In [10]:
import gym
import numpy as np
from tqdm import tqdm
import torch
import wandb

from fractal_zero.config import FMCConfig
from fractal_zero.search.fmc import FMC
from fractal_zero.models.prediction import FullyConnectedPredictionModel
from fractal_zero.vectorized_environment import (
    RayVectorizedEnvironment,
    VectorizedDynamicsModelEnvironment,

)
from fractal_zero.trainers.online import OnlineFMCPolicyTrainer

from fractal_zero.tests.test_vectorized_environment import build_test_joint_model

In [11]:
NUM_WALKERS = 64

class CartpolePolicy(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.net = torch.nn.Sequential(
            torch.nn.Linear(4, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 16),
            torch.nn.ReLU(),
            torch.nn.Linear(16, 2),
        )

    def forward(self, observations, with_randomness: bool = False, argmax: bool = True):
        observations = torch.tensor(observations).float()

        y = self.net(observations)

        if with_randomness:
            # center = embeddings.std()
            center = y.var()
            centered_uniform_noise = (torch.rand_like(y) * center) - (center / 2)
            y += centered_uniform_noise

        # TODO: refac
        if argmax:
            return torch.argmax(y, dim=-1)
        return y

    def parse_actions(self, actions):
        return actions.tolist()

policy_model = CartpolePolicy()

In [12]:
# optimizer = torch.optim.SGD(policy_model.parameters(), lr=0.01, weight_decay=1e-4)
optimizer = torch.optim.Adam(policy_model.parameters(), lr=0.01, weight_decay=1e-4)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)
policy_trainer = OnlineFMCPolicyTrainer("CartPole-v0", policy_model, optimizer, NUM_WALKERS)

/home/nollyd/anaconda3/envs/gailzero/lib/python3.10/site-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/home/nollyd/anaconda3/envs/gailzero/lib/python3.10/site-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/home/nollyd/anaconda3/envs/gailzero/lib/python3.10/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
2022-09-27 02:05:16,678	WARNING worker.py:1829 -- WARNING: 120 PYTHON worker processes have been started on node: 73202a2786b

In [13]:
wandb.init(project="fz-policy-trainer-game-tree")

/home/nollyd/anaconda3/envs/gailzero/lib/python3.10/site-packages/wandb/sdk/lib/ipython.py:46: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display  # type: ignore


(pid=2066789) WARNING: CPU random generator seem to be failing, disabling hardware random number generation
(pid=2066789) WARNING: RDRND generated: 0xffffffff 0xffffffff 0xffffffff 0xffffffff
(pid=2066778) WARNING: CPU random generator seem to be failing, disabling hardware random number generation
(pid=2066778) WARNING: RDRND generated: 0xffffffff 0xffffffff 0xffffffff 0xffffffff
(pid=2066774) WARNING: CPU random generator seem to be failing, disabling hardware random number generation
(pid=2066774) WARNING: RDRND generated: 0xffffffff 0xffffffff 0xffffffff 0xffffffff
(pid=2066775) WARNING: CPU random generator seem to be failing, disabling hardware random number generation
(pid=2066775) WARNING: RDRND generated: 0xffffffff 0xffffffff 0xffffffff 0xffffffff
(pid=2066781) WARNING: CPU random generator seem to be failing, disabling hardware random number generation
(pid=2066781) WARNING: RDRND generated: 0xffffffff 0xffffffff 0xffffffff 0xffffffff
(pid=2066788) WARNING: CPU random genera

eval/total_rewards,▁▆██████████████████████████████████████
train/epsiode_reward,███████████▆█████████████████▁██████████
train/loss,▃▃▂▁▃▃▅▅▆▁▂▃▄▁▁▃▃▁▂▃▃▄▂▂▂▄▄█▂▃▂▁▂▅▂▂▃▁▅▃
eval/total_rewards,32.0
train/epsiode_reward,32.0
train/loss,0.69525


(_RayWrappedEnvironment pid=2066784) /home/nollyd/anaconda3/envs/gailzero/lib/python3.10/site-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
(_RayWrappedEnvironment pid=2066784)   logger.warn(
(_RayWrappedEnvironment pid=2066784) /home/nollyd/anaconda3/envs/gailzero/lib/python3.10/site-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
(_RayWrappedEnvironment pid=2066784)   deprecation(
(_RayWrappedEnvironment pid=2066784) /home/nollyd/anaconda3/envs/gailzero/lib/python3.10/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. T

(_RayWrappedEnvironment pid=2066819) /home/nollyd/anaconda3/envs/gailzero/lib/python3.10/site-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
(_RayWrappedEnvironment pid=2066819)   logger.warn(
(_RayWrappedEnvironment pid=2066819) /home/nollyd/anaconda3/envs/gailzero/lib/python3.10/site-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
(_RayWrappedEnvironment pid=2066819)   deprecation(
(_RayWrappedEnvironment pid=2066819) /home/nollyd/anaconda3/envs/gailzero/lib/python3.10/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. T

In [14]:
num_episodes = 100
train_steps_per_episode = 2
eval_every = 20
max_steps = 200

for _ in range(num_episodes):
    policy_trainer.generate_episode_data(max_steps)

    for i in range(train_steps_per_episode):
        policy_trainer.train_on_latest_episode()

        if i % eval_every == 0:
            policy_trainer.evaluate_policy(max_steps)

    lr_scheduler.step()

(_RayWrappedEnvironment pid=2066973) /home/nollyd/anaconda3/envs/gailzero/lib/python3.10/site-packages/gym/envs/classic_control/cartpole.py:179: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
(_RayWrappedEnvironment pid=2066973)   logger.warn(
(_RayWrappedEnvironment pid=2066775) /home/nollyd/anaconda3/envs/gailzero/lib/python3.10/site-packages/gym/envs/classic_control/cartpole.py:179: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
(_RayWrappedEnvironment pid=2066775)   logger.warn(
(_RayWrappedEnvironment pid=2067611) /home/nollyd/anaconda3/envs/gailzero/lib/python3.10/site-packages/gym/envs/classic_control/cartpole.py:179: UserW

In [ ]:
policy_trainer.fmc.clone_receives

In [ ]:
policy_trainer.fmc.tree.render()

In [ ]:
str(policy_trainer.fmc.tree.g)